In [1]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [2]:
URM_test=sps.load_npz("./dressipi_dataset/URM_test_final.npz")

In [3]:
URM_test.shape

(4439967, 23690)

In [4]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [5]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [6]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [7]:
is_sorted(Session_ids)

True

In [8]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF", "TopPop15"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./final_ranker_dataset/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
15960971
loaded UCF
0
loading ICBF
14855475
loaded ICBF
0
loading Graph
8321446
loaded Graph
0
loading ICF
17771428
loaded ICF
0
loading TopPop15
7500000
loaded TopPop15


In [9]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
map_ids["Real_Id"]=Session_ids

In [10]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [11]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
import ast
submission_df=pd.read_csv("./candidate-nn/final_candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [13]:
import ast
submission_df_tr=pd.read_csv("PATH_TO_TRANSFORMERS_CANDIDATES/final_candidates_transformer.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [17]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr

In [18]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [19]:
dataset.fillna(0,inplace=True)

In [20]:
gc.collect()

2189

In [21]:
# dataset_macro_features = pd.read_csv('./macro-feats/test_final.csv',
#                                      usecols=['session_id',
#                                                 'session_similarity',
#                                                 'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
#                                               'length_of_session_seconds',
#                    'avg_time_spent_per_item_seconds',
#                    'variance_time_spent_per_item_seconds',
#                    'n_seen_items',
#                    'n_unique_items',])
#
# dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
# dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [23]:
dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [24]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Score_TopPop15,Session_Id,score,rank,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF
0,11,0.035145,0.115033,0.006554,0.311263,0.000000,61,0.000000,101,239.07347,3.357151,12.915223,0.563566
1,19,0.018681,0.000000,0.000000,0.000000,0.301016,61,0.000461,95,239.07347,3.357151,12.915223,0.563566
2,23,0.039035,0.072503,0.007389,0.325638,0.000000,61,0.000000,101,239.07347,3.357151,12.915223,0.563566
3,25,0.025481,0.081094,0.004554,0.006163,0.000000,61,0.000000,101,239.07347,3.357151,12.915223,0.563566
4,69,0.056301,0.085104,0.011284,0.338065,0.000000,61,0.001512,26,239.07347,3.357151,12.915223,0.563566


In [25]:
gc.collect()

0

In [26]:
dataset=dataset.sort_values("Session_Id")

In [27]:
gc.collect()

0

In [28]:
import lightgbm

In [29]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./{k}_fold/model_{index}.txt')
    dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.mean(x),axis=1)
#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [31]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

4

In [32]:
dataset.head()

,Session_Id,Item_ID,score
0,61,11,-2.218253
28700479,61,446,-3.776089
28700478,61,356,-2.643857
28700477,61,323,-3.567577
28700476,61,320,-4.483993


In [33]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [34]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [35]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [36]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 446, 356, 323, 320, 270, 263, 232, 513, 2...
score         [-2.218252866012813, -3.7760893640223365, -2.6...
indices       [420, 543, 478, 430, 499, 383, 531, 520, 347, ...
Name: 0, dtype: object

In [37]:
dataset.iloc[0]["Item_ID"][536]

4586

In [38]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [39]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [40]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [41]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 4574, 1563, 143, 3581, 3251, 4436, 3414...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 3778, 143, 4655, 1683, 1608, 4148, 3550...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 2704, 4146, 1936, 4162, 2330, 3827, 733...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[4202, 3338, 2769, 16, 4990, 2180, 3026, 2785,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2970, 2899, 3599, 3441, 2471, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 191, 3424, 1468, 4026, 2720, 3037, 3656,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 4586, 3505, 3550, 2423, 535...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 779, 3247, 4966, 3116, 819, 1906, 3879,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 2285, 1972, 4740, 4220, 750, 3414, 292, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [42]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [43]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [44]:
dataset.columns=["session_id","code","rank"]

In [45]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 4574, 1563, 143, 3581, 3251, 4436, 3414...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 3778, 143, 4655, 1683, 1608, 4148, 3550...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 2704, 4146, 1936, 4162, 2330, 3827, 733...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(4202, 3338, 2769, 16, 4990, 2180, 3026, 2785,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2970, 2899, 3599, 3441, 2471, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [46]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [47]:
mapping= pd.read_csv('./dressipi_dataset/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [48]:
print(len(submission_df))

5000000


In [49]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [50]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [51]:
submission_df.to_csv(
    f'predictions_final_{k}_fold.csv',
    index=False
    )